In [4]:
import pandas as pd
import numpy as np
import ast

In [2]:
yelp2 = pd.read_pickle("yelp_2.p")
rubmaps = pd.read_csv("rubmaps_final.csv")

In [3]:
yelp2["date"] = yelp2['review{}.datePublished'].str.split("\n")
yelp2["date"] = yelp2["date"].apply(lambda x: [pd.to_datetime(i) for i in x] if type(x) == list else x)

In [5]:
yelp2["date_diff"] = yelp2["date"].apply(lambda x: max(x) - min(x) if type(x) == list else x)

In [6]:
yelp2["range_days"] = yelp2["date_diff"].dt.days

In [7]:
yelp2 = yelp2.drop(["@type", "_time", "verifiedLicenseInfo.licenses{}.issuedBy",
            "verifiedLicenseInfo.licenses{}.licenseNumber",
            "verifiedLicenseInfo.licenses{}.licensee",
           "verifiedLicenseInfo.licenses{}.trade",
           "verifiedLicenseInfo.licenses{}.verifiedDate",
           "verifiedLicenseInfo.bizSiteUrl", "verifiedLicenseInfo.licenses{}.expiryDate",
           "snippet.readMoreUrl", "snippet.thumbnail.src", "snippet.thumbnail.srcset",
           "searchResultBusinessHighlights.businessHighlights{}.title", 
           "searchResultLayoutType", "searchResultLayoutType",
           "searchResultBusinessHighlights.businessHighlights{}.iconName", 
           "searchResultBusinessHighlights.businessHighlights{}.id",
           "searchResultBusinessHighlights.businessHighlights{}.bizPageIconName",
           "searchResultBusinessHighlights.bizSiteUrl"], axis = 1)

## Fix lat and lon

In [8]:
yelp2["lat_lon"] = yelp2.lat.astype(str) + ", " + yelp2.long.astype(str)
rubmaps["lat_lon"] = rubmaps["geo.lat"].astype(str) + ", " + rubmaps["geo.lon"].astype(str)

In [9]:
yelp2["lon"] = round(yelp2["long"].astype(float),4)
yelp2["lat_2"] = yelp2["lat"].astype(float)

In [10]:
yelp2[yelp2["lat"].astype(float) > 41.995237]

,address,address.addressCountry,address.addressLocality,address.addressRegion,address.postalCode,address.streetAddress,address_administrative_area_level_1,address_administrative_area_level_2,address_country,address_formatted_address,...,snippet.text,telephone,lat,long,date,date_diff,range_days,lat_lon,lon,lat_2
5027,"365 S Mountain Ave 3&4, CA, 91786",US,Upland,CA,91786,365 S Mountain Ave 3&4,NaN,NaN,NaN,NaN,...,NaN,1.909962e+10,42.1881244,-122.6995545,"[2019-12-26 00:00:00, 2019-10-10 00:00:00, 201...",718 days,718.0,"42.1881244, -122.6995545",-122.6996,42.188124
27906,"731 S Hwy 101, CA, 94513",US,Solana,CA,94513,731 S Hwy 101,NaN,NaN,NaN,NaN,...,NaN,1.505469e+10,46.9057608,-123.7170983,[2017-08-30 00:00:00],0 days,0.0,"46.9057608, -123.7170983",-123.7171,46.905761
39865,"Biatchi Salon & Spa 299 W Foothill Blvd , Ste ...",US,Upland,CA,91786,"Biatchi Salon & Spa 299 W Foothill Blvd , Ste 100",NaN,NaN,NaN,NaN,...,NaN,1.909490e+10,42.5978176,-82.9547448,"[2016-07-10 00:00:00, 2015-02-24 00:00:00]",502 days,502.0,"42.5978176, -82.9547448",-82.9547,42.597818


In [11]:
yelp2[(yelp2["lon"].astype(float) > -114.133039) & yelp2['address'].notnull()][['address', 'name', 'address.addressLocality']]

,address,name,address.addressLocality
1357,"4 & 6 Wildwood Ave, CA, 94610",Weeds Salon,Oakland
3489,"Sunrise Blvd & Greenback Ln, CA, 95610",Salon Ooh Lala,Citrus Heights
4646,"632 E 14th St Kj’s Barbershop, CA, 94577",Mommy Who Cuts Hair,San Leandro
5587,"Chess & Burman Salons 11239 Ventura Blvd, 21...",Girl Talk Natural Nails and Foot Reflexology,Studio City
7226,"Pico & La Cienega, CA, 90035",Sports & Neuromuscular Massage Therapy,Los Angeles
7911,"707 10th Ave Ste 219 Loft Salon, CA, 92101",Eye Candy Salon,San Diego
8563,"One Southland Mall Drive #5528, CA, 94545",Shapes Brow Bar Cart,Hayward
8564,111 Town & Country Dr Ste G Tribez Salon & Spa...,Skin Care By Zakiya,Danville
9956,1022 North Hollywood Way Darling Nikki’s Salon...,Dead Sexy Hair by Brittney Lee,Burbank
12402,"Geary & 16th Ave, CA, 94118","Amy Seo, CMT",San Francisco


In [12]:
# Fix lat long for Face Appeal Beauty Studio
yelp2.at[5027, 'lat_2'] = 34.089323
yelp2.at[5027, 'lon'] = -117.671584
#Fix lat lon for Sound Touch
yelp2.at[27906, 'lat_2'] = 32.982086
yelp2.at[27906, 'lon'] = -117.270014
#Fix lat lon for Biatchi, even tho its closed permenently
yelp2.at[39865, 'lat_2'] = 34.108276
yelp2.at[39865, 'lon'] = -117.655003
#Fix lat lon for Weeds Salon
yelp2.at[1357, 'lat_2'] = 37.818698
yelp2.at[1357, 'lon'] = -122.244440
#Fix lat lon for Salon Ooh Lala STREET CORNER
yelp2.at[3489, 'lat_2'] = 38.678489
yelp2.at[3489, 'lon'] = -121.271611
# Fix lat lon for Mommy Who Cuts Hair
yelp2.at[4646, 'lat_2'] = 37.730615
yelp2.at[4646, 'lon'] = -122.160087
#Fix for Girl Talk Natural Nails and Foot Reflexology
yelp2.at[5587, 'lat_2'] = 34.142008
yelp2.at[5587, 'lon'] = -118.375033
#Fix for Sports & Neuromuscular Massage Therapy STREET CORNER
yelp2.at[7226, 'lat_2'] = 34.053345
yelp2.at[7226, 'lon'] = -118.376156
#FIX FOR Eye Candy Salon
yelp2.at[7911, 'lat_2'] = 32.712866
yelp2.at[7911, 'lon'] = -117.155289

In [13]:
# skin care by zakiya
yelp2.at[8564, 'lat_2'] = 37.814305
yelp2.at[8564, 'lon'] = -121.998357
# shapes brow bar cart
yelp2.at[8563, 'lat_2'] = 37.6506802
yelp2.at[8563, 'lon'] = -122.1034966
# dead sexy hair
yelp2.at[9956, 'lat_2'] = 34.168680
yelp2.at[9956, 'lon'] = -118.347139
# Amy Seo, CMT
yelp2.at[12402, 'lat_2'] = 37.780475
yelp2.at[12402, 'lon'] = -122.475116
# Studio 56 STREET CORNER AND PERMAMNETLY CLOSED
yelp2.at[13464, 'lat_2'] = 36.555646
yelp2.at[13464, 'lon'] = -121.920973
# CHARLIE BENNETT STREST CORNES
yelp2.at[15587, 'lat_2'] = 37.808687
yelp2.at[15587, 'lon'] = -122.252770
# Joe Anthony Massage & Bodywork
yelp2.at[15451, 'lat_2'] = 37.752808
yelp2.at[15451, 'lon'] = -122.409141
# healey touch
yelp2.at[15776, 'lat_2'] = 37.533407
yelp2.at[15776, 'lon'] = -121.920101
# Bodywork by Egie
yelp2.at[15822, 'lat_2'] = 34.019191
yelp2.at[15822, 'lon'] = -118.478140
# jennfier
yelp2.at[17959, 'lat_2'] = 37.814305
yelp2.at[17959, 'lon'] = -121.998357
# Hair by Courtney Canaday
yelp2.at[17724, 'lat_2'] = 33.914884
yelp2.at[17724, 'lon'] = -117.441880
# salon ciera
yelp2.at[22049, 'lat_2'] = 37.973316
yelp2.at[22049, 'lon'] = -122.526169
# Raeann Venard Beauty Professional
yelp2.at[22147, 'lat_2'] = 39.750379
yelp2.at[22147, 'lon'] = -121.854467
# aviante health
yelp2.at[25342, 'lat_2'] = 34.180101
yelp2.at[25342, 'lon'] = -118.864126

In [14]:
# The Genesis Journy STREET CORNER
yelp2.at[25551, 'lat_2'] = 38.705990
yelp2.at[25551, 'lon'] = -121.327047
# winzer hair salon
yelp2.at[29912, 'lat_2'] = 33.821716
yelp2.at[29912, 'lon'] = -116.546023
yelp2.at[25551, 'lon'] = -121.327047
# sassy nail
yelp2.at[34286, 'lat_2'] = 38.730407
yelp2.at[34286, 'lon'] = -120.786480
# profiles hair studio
yelp2.at[34741, 'lat_2'] = 36.554461
yelp2.at[34741, 'lon'] = -121.923155
# Massage by shelley
yelp2.at[35635, 'lat_2'] = 39.763591
yelp2.at[35635, 'lon'] = -121.868634
# Skincare By Melissa
yelp2.at[37238, 'lat_2'] = 33.203381
yelp2.at[37238, 'lon'] = -117.240730
# Wellness By Candy
yelp2.at[39728, 'lat_2'] = 32.721069
yelp2.at[39728, 'lon'] = -117.165053

In [15]:
# zen temple
yelp2.at[41139, 'lat_2'] = 37.806205
yelp2.at[41139, 'lon'] = -122.275947
# ANGEL TOUCH THERAPUTIC MASSAGE IS PROLLY GONNA GET DROPPED
# Excellent Touch Massage Therapy
yelp2.at[43681, 'lat_2'] = 35.374109
yelp2.at[43681, 'lon'] = -119.013835
# best blowouts
yelp2.at[44010, 'lat_2'] = 32.602221
yelp2.at[44010, 'lon'] = -117.064207
# Domani Salon of Beauty & Wellness
yelp2.at[46511, 'lat_2'] = 32.682700
yelp2.at[46511, 'lon'] = -117.177818
# bay beauty
yelp2.at[54479, 'lat_2'] = 35.369168
yelp2.at[54479, 'lon'] = -120.854049

In [16]:
yelp_revs = pd.read_csv("yelp_reviews_list.csv", index_col=0)

In [17]:
yelp2 = yelp2.join(yelp_revs)

In [19]:
yelp2 = yelp2[yelp2.address.notna()]

In [20]:
yelp2["lat_lon"] = yelp2.lat_2.astype(str) + ", " + yelp2.lon.astype(str)
rubmaps["lat_lon"] = rubmaps["geo.lat"].astype(str) + ", " + rubmaps["geo.lon"].astype(str)

In [21]:
yelp2["yelp"] = 1
yelp2["rubmaps"] = 0
rubmaps["yelp"] = 0
rubmaps["rubmaps"] = 1

In [22]:
yelp2['reviews'] = yelp2['reviews'].apply(lambda x: ast.literal_eval(x))

In [23]:
yelp2 = yelp2.explode('reviews')

In [41]:
yelp = yelp2[['name', "address", "reviews","range_days",
              'lat_2', 'lon', 'yelp', 'rubmaps']].rename(columns={'lat_2': 'lat', "reviews":"review"})

## Fix null lat and lon for yelp

In [42]:
y_null = yelp[yelp["lat"].isna()]
#y_null.to_csv("yelp_null.csv")

In [43]:
null_indexes = list(y_null.index)
yelp = yelp.drop(null_indexes)

In [44]:
y_fixed = pd.read_csv("completed_yelp.csv")
y_fixed = y_fixed.set_index("Unnamed: 0")
y_fixed.index.name = None

In [45]:
y_fixed[y_fixed["lat"] == -1]

,name,address,lat,lon,yelp,rubmaps
19735,"Adam Atman, LAc, MMQ,CMT - Los Gatos Acupunctu...","200 S Santa Cruz Ave​, CA, 95030",-1.0,-1.0,1,0
35939,Red Persimmon Nails & Spa,"1633 S Alameda St, CA, 90220",-1.0,-1.0,1,0
40037,Kymm Ann Wallin,"Mother's Journey, CA, 95060",-1.0,-1.0,1,0
47460,Christians Wonderful Massage,"12301 Excalibur St, CA,",-1.0,-1.0,1,0
54944,Know Knots Massage,"​5015 Canyon Crest Dr, CA, 92507",-1.0,-1.0,1,0
56777,Know Knots Massage,"​5015 Canyon Crest Dr, CA, 92507",-1.0,-1.0,1,0


In [46]:
# Adam Atman
y_fixed.at[19735, 'lat'] = 37.220500
y_fixed.at[19735, 'lon'] = -121.986011
# Red Persimmon Nails & Spa
y_fixed.at[35939, 'lat'] = 33.876055 
y_fixed.at[35939, 'lon'] = -118.219873
#Know Knots Massage
y_fixed.at[54944, 'lat'] = 33.959429
y_fixed.at[54944, 'lon'] = -117.331608
y_fixed.at[56777, 'lat'] = 33.959429
y_fixed.at[56777, 'lon'] = -117.331608
y_fixed = y_fixed.drop([40037,47460])

In [47]:
yelp = pd.concat([yelp, y_fixed], sort=True)

## Concat!

In [48]:
rub =rubmaps[['name{}', 'address', "more_details", 'geo.lat', 'geo.lon',
              'yelp', 'rubmaps']].rename(columns={'name{}': 'name','geo.lat':'lat',
                                                  'geo.lon':'lon', "more_details":"review"})

In [49]:
rub.columns

Index(['name', 'address', 'review', 'lat', 'lon', 'yelp', 'rubmaps'], dtype='object')

In [50]:
yelp.columns

Index(['address', 'lat', 'lon', 'name', 'range_days', 'review', 'rubmaps',
       'yelp'],
      dtype='object')

In [54]:
concat = pd.concat([yelp, rub], sort=True)
concat = concat.reset_index().drop(["index"], axis = 1)

In [55]:
concat["lon"] = round(concat["lon"].astype(float),5)
concat["lat"] = round(concat["lat"].astype(float),5)
concat['lat_lon'] = list(zip(concat['lat'], concat['lon']))

In [56]:
import pickle
pickle.dump(concat, open( "concat.p", "wb" ))

In [34]:
def similar(t1, t2, epsilon):
    diff1 = abs(t1[0] - t2[0])
    diff2 = abs(t1[1] - t2[1])
    if (diff1 <= epsilon) and (diff2 <= epsilon):
        return True
    else:
        return False

In [35]:
def fuzzy_latlon(df, epsilon=0.0001):
    df_new = df.sort_values(by=['lat','lon'])
    df_new = df_new.reset_index().drop(["index"], axis = 1)
    g_id = 0
    df_new['g_id'] = -1
    j = 0
    for i in range(len(df) - 1):
        if i >= j:
            old = df_new.iloc[i]['lat_lon']
            df_new.at[i, 'g_id'] = g_id
            j = i + 1
            while similar(df_new.iloc[j]['lat_lon'],old,epsilon):
                df_new.at[j, 'g_id'] = g_id
                j += 1
            g_id += 1
        if i % 10000 == 0:
            print(i)
    return df_new

In [36]:
fuz_concat = fuzzy_latlon(concat)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000


In [37]:
fuz_grouped = fuz_concat.groupby(['g_id']).agg({"lat":'max', "lon":'max', "yelp": "sum",
                                  "rubmaps":"sum", "name": lambda x: list(set(x)), "address":lambda x: list(set(x))})
fuz_grouped

,lat,lon,yelp,rubmaps,name,address
g_id,,,,,,
-1,45.50375,-73.56947,1,0,[Hands2home Mobile Massage],"[Mobile, CA]"
0,30.52796,-99.85290,1,0,[Angel Touch Therapeutic Massage],"[Between I-10 And 210 Fwy, CA, n]"
1,31.02507,-114.84080,1,0,[Massage Therapy & Prenatal],"[San felipe & Yerba Buena Road, CA, 95121]"
2,32.54368,-117.04650,1,0,[Ulta Beauty],"[3951 Camino De La Plz Plaza at the Border, CA..."
3,32.54396,-117.03750,1,0,[Amerinails],"[4509 Camino De La Plz Ste 364, CA, 92173]"
...,...,...,...,...,...,...
41208,41.77279,-124.20410,1,0,[Your Salon],"[118 W Washington Blvd, CA, 95531]"
41209,41.78027,-124.19910,1,0,[Sherman Deborah Certified Massage Therapist],"[1910 Northcrest Dr Ste C, CA, 95531]"
41210,41.78139,-124.19820,1,0,[EYECANDY],"[1995 Northcrest Dr, CA, 95531]"


In [38]:
fuz_grouped[(fuz_grouped['yelp'] > 0) & (fuz_grouped['rubmaps'] > 0)]

,lat,lon,yelp,rubmaps,name,address
g_id,,,,,,
208,32.65044,-117.05140,2,6,"[Arcadia Massage Spa, Arcadia Massage Spa\nChi...","[3404 Bonita Rd, CA, 91910, 3404 Bonita Rd Chu..."
408,32.71100,-117.16220,2,11,"[Beijing Foot Massage, Gaslamp Spa]","[540 3rd Ave, CA, 92101, 540 Third Ave, CA, 92..."
434,32.71273,-117.16077,1,21,[Bangkok Day Spa],"[711 4th Ave Ste 201, CA, 92101, 711 Fourth Av..."
445,32.71345,-117.15712,2,35,"[Orchid Thai Massage & Spa, Rama Thai Massage]","[801 F St, CA, 92101, 801 F St San Diego, CA, ..."
652,32.73764,-117.17590,1,56,"[BaySide Spa, Bayside Spa]","[1430 Sassafras St San Diego, CA, 92103, 1430 ..."
...,...,...,...,...,...,...
40882,40.56408,-122.34380,4,4,[Finelines Skincare Rejuvenation Center & Perm...,"[1484 Hartnell Ave Suite, F, CA, 96002, 1484 H..."
40892,40.56622,-122.36200,1,7,[Lin Asia Spa],"[2833 Bechelli Ln, CA, 96002, 2833 bechelli Ln..."
40964,40.57830,-122.35828,1,12,"[Hong's Acupressure Body Massage, Red Chair Sa...","[2004 Hilltop Dr, CA, 96002, 2002 Hilltop Dr R..."


In [39]:
fuz_grouped.to_csv("unique_bus.csv")

In [ ]:
fuz_grouped = fuz_concat.groupby(['g_id']).sum()
fuz_grouped[(fuz_grouped['yelp'] > 0) & (fuz_grouped['rubmaps'] > 0)]
fuz_grouped['reviews'] = fuz_grouped['yelp'] + fuz_grouped['rubmaps']
#fuz_grouped[fuz_grouped['reviews'] > 1]

In [ ]:
(fuz_concat["lat"] == 0).sum()

In [ ]:
grouped = concat.groupby(['lat', 'lon']).sum()

#grouped[(grouped['yelp'] > 0) & (grouped['rubmaps'] > 0)]
grouped['reviews'] = grouped['yelp'] + grouped['rubmaps']
grouped[grouped['reviews'] > 2]

In [ ]:
concat.groupby(['address']).count()

In [ ]:
concat.groupby(['name']).count()

In [ ]:
yelp2.sort_values(by=['lon'], inplace=True)
rubmaps.sort_values(by=['geo.lon'],inplace=True)

In [ ]:
yelp2_lon = yelp2[yelp2['lon'].notna()]

In [ ]:
merged_lon = pd.merge_asof(yelp2_lon, rubmaps, left_on='lon', right_on='geo.lon', direction='nearest')

In [ ]:
merged_lon_filtered = merged_lon.loc[abs(merged_lon['geo.lon'] - merged_lon['lon']) < 0.0001, :]#[['geo.lon', 'lon']]

In [ ]:
#merged[['geo.lon', 'lon','geo.lat', 'lat']]
merged1 = merged_lon_filtered.loc[
    abs(merged_lon_filtered['geo.lat'] - merged_lon_filtered['lat_2']) < 0.0001,:]

In [ ]:
yelp2.sort_values(by=['lat_2'], inplace=True)
rubmaps.sort_values(by=['geo.lat'],inplace=True)

In [ ]:
yelp2_lat = yelp2[yelp2['lat_2'].notna()]

In [ ]:
merged_lat = pd.merge_asof(yelp2_lat, rubmaps, left_on='lat_2', right_on='geo.lat', direction='nearest')

In [ ]:
merged_lat_filtered = merged_lat.loc[abs(merged_lat['geo.lat'] - merged_lat['lat_2']) < 0.0001, :]#[['geo.lon', 'lon']]
#= np.nan
#merged.dropna()

In [ ]:
#merged[['geo.lon', 'lon','geo.lat', 'lat']]
merged2 = merged_lat_filtered.loc[
    abs(merged_lat_filtered['geo.lon'] - merged_lat_filtered['lon']) < 0.0001,:]

In [ ]:
pd.concat([merged1, merged2]).sort_index()